In [1]:
import pandas as pd
path = ''

### Set Path

In [6]:
dimstore_path = path+'DimStore.xlsx'
store_rank = path+'StoreRanking.xlsx'
transaction_his = path+'TransactionHistory (1).txt'
zip_code = path+'us_zipcodes.xlsx'

### Read excel file

In [7]:
def read_xlsx(filepath):
    fr = open(filepath,'rb')
    df = pd.read_excel(fr)
    fr.close()
    return df

In [8]:
dim_df = read_xlsx(dimstore_path)
store_rank_df = read_xlsx(store_rank)
zip_code_df = read_xlsx(zip_code)

### Read txt file

In [9]:
fr = open(transaction_his,'rb')
str1_list = [str(line).replace("'",'').replace('b\\xef\\xbb\\xbf','').replace('\\r\\n','').replace('"','') for line in fr.readlines()]
fr.close()
str2_list = [x.split('\\t') for x in str1_list] 
transaction_his_df = pd.DataFrame(str2_list[1:])
transaction_his_df.columns = str2_list[0]
transaction_his_df['TheDate'] = [x.replace('b','') for x in transaction_his_df['TheDate']]


### Relate each File-- Confirm the Merging logistic among datasets

In [10]:
dim_df.columns
###Delete'FTVolume', 'APVolume', 'ACVolume'
dim_df.drop(['FTVolume', 'APVolume', 'ACVolume'],axis=1,inplace=True)


In [11]:
#see channel
result1 = dim_df[dim_df['Channel']=='Web']

In [13]:
#write csv
result1.to_csv('/Users/tianyizhao/Desktop/PUMA Dataset/channel_web.csv',index=0)

In [14]:
result1

,InternationalStoreCode,NickName,ZipCode,CloseDt,Channel,RegionName,DistrictName,AreaName,TimeZone,LocationType,SellingSqFt,SimilarStore
11,USW01,Puma.com,1886,20991231.0,Web,NaN,NaN,NaN,PST,Internet,NaN,NaN
70,USRE0004,EBay,NaN,20991231.0,Web,EAST,North East,NaN,CST,WHS,NaN,NaN
71,USRE0005,Jet.com,NaN,20991231.0,Web,EAST,North East,NaN,CST,WHS,NaN,NaN
72,USRE0006,Rakuten,NaN,20991231.0,Web,EAST,North East,NaN,CST,WHS,NaN,NaN
218,USW02,Tretorn.com,1886,20160210.0,Web,NaN,NaN,NaN,PST,Internet,NaN,NaN


In [15]:
dim_df['Channel'].drop_duplicates()

0      Outlet
11        Web
14    Concept
30        WHS
32        NaN
Name: Channel, dtype: object

In [16]:
store_rank_df.columns

Index(['InternationalStoreCode', 'Rank'], dtype='object')

In [17]:
store_rank_df['Rank'].drop_duplicates()

0       A
19     AA
28    AAA
36      B
70      C
Name: Rank, dtype: object

In [18]:
zip_code_df.columns

Index(['Type', 'ZipCode', 'PrimaryRecord', 'Population',
       'HouseholdsPerZipcode', 'WhitePopulation', 'BlackPopulation',
       'HispanicPopulation', 'AsianPopulation', 'HawaiianPopulation',
       'IndianPopulation', 'OtherPopulation', 'MalePopulation',
       'FemalePopulation', 'PersonsPerHousehold', 'AverageHouseValue',
       'IncomePerHousehold', 'MedianAge', 'MedianAgeMale', 'MedianAgeFemale',
       'Latitude', 'Longitude', 'Elevation', 'State', 'StateFullName',
       'CityType', 'CityAliasAbbreviation', 'AreaCode', 'City',
       'CityAliasName', 'County', 'CountyFIPS', 'StateFIPS', 'TimeZone',
       'DayLightSaving', 'MSA', 'PMSA', 'CSA', 'CBSA', 'CBSA_DIV', 'CBSA_Type',
       'CBSA_Name', 'MSA_Name', 'PMSA_Name', 'Region', 'Division',
       'MailingName', 'NumberOfBusinesses', 'NumberOfEmployees',
       'BusinessFirstQuarterPayroll', 'BusinessAnnualPayroll',
       'BusinessEmploymentFlag', 'GrowthRank', 'GrowthHousingUnits2009',
       'GrowthHousingUnits2010', 'G

In [19]:
transaction_his_df.columns

Index(['TheDate', 'InternationalStoreCode', 'Transaction`Type',
       'ShipToZipCode', 'Transactions'],
      dtype='object')

In [20]:
df1 = pd.merge(store_rank_df,transaction_his_df,how='left',on='InternationalStoreCode')

In [21]:
df2 = pd.merge(df1,dim_df,how='left',on='InternationalStoreCode')

In [23]:
df3 = pd.merge(df2,zip_code_df,how='left',on='ZipCode')

In [24]:
df4 = df3[(df3['Rank']=='AAA')|(df3['Rank']=='C')]

In [25]:
df4.shape

(2928904, 104)

In [26]:
label = df4['Rank'].apply(lambda x: 1 if x=='AAA' else 0)

In [27]:
df4.drop(['Rank'],axis=1,inplace=True)

/Users/tianyizhao/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [28]:
df4['label'] = label

/Users/tianyizhao/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [30]:
df4.to_csv('/Users/tianyizhao/Desktop/PUMA Dataset/mid_result.csv',index=0)

In [36]:
import pandas as pd
fr = open('/Users/tianyizhao/Desktop/PUMA Dataset/mid_result.csv')
df4 = pd.read_csv(fr)

/Users/tianyizhao/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (3,6,9,10,11,15,41,42,58,80,81,83) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [37]:
df4.columns

Index(['InternationalStoreCode', 'TheDate', 'Transaction`Type',
       'ShipToZipCode', 'Transactions', 'NickName', 'ZipCode', 'CloseDt',
       'Channel', 'RegionName',
       ...
       'StateANSI', 'CountyANSI', 'ZIPIntroDate', 'AliasIntroDate',
       'FacilityCode', 'CityDeliveryIndicator', 'CarrierRouteRateSortation',
       'FinanceNumber', 'UniqueZIPName', 'label'],
      dtype='object', length=104)

In [ ]:
discrete_list = []
continous_list = []
for x in df4.columns:
    if df4[x].dtypes=='object': 
        if len(df4[x].drop_duplicates())<=10:
            discrete_list.append(x)
    else:
        continous_list.append(x)

In [39]:
df5 = df4.copy()

In [40]:
for x in discrete_list:
    df5[x] = df5[x].fillna('unknown')

In [41]:
for x in discrete_list:
    print(x)
    print(df5[x].drop_duplicates())

Transaction`Type
0         Sale
2       Return
5859    unkown
Name: Transaction`Type, dtype: object
Channel
0          Outlet
5859      Concept
122669        Web
Name: Channel, dtype: object
RegionName
0           EAST
2490        WEST
122669    unkown
Name: RegionName, dtype: object
AreaName
0                unkown
11816        West Coast
25043          Mid West
53443     North Central
55756     South Central
77507        South East
115304       North East
Name: AreaName, dtype: object
TimeZone_x
0        EST
2490     MST
5075     PST
18906    CST
Name: TimeZone_x, dtype: object
LocationType
0             Mall
5859        Street
111944      unkown
122669    Internet
Name: LocationType, dtype: object
PrimaryRecord
0       unkown
5075         P
Name: PrimaryRecord, dtype: object
CityType
0         unkown
5075           P
11816          Z
120826         B
Name: CityType, dtype: object
CityAliasAbbreviation
0                unkown
77507     WEST PALM BCH
95645      ST AUGUSTINE
120826    

In [42]:
continous_list

['Transactions',
 'CloseDt',
 'SellingSqFt',
 'Type',
 'Population',
 'HouseholdsPerZipcode',
 'WhitePopulation',
 'BlackPopulation',
 'HispanicPopulation',
 'AsianPopulation',
 'HawaiianPopulation',
 'IndianPopulation',
 'OtherPopulation',
 'MalePopulation',
 'FemalePopulation',
 'PersonsPerHousehold',
 'AverageHouseValue',
 'IncomePerHousehold',
 'MedianAge',
 'MedianAgeMale',
 'MedianAgeFemale',
 'Latitude',
 'Longitude',
 'Elevation',
 'CountyFIPS',
 'StateFIPS',
 'TimeZone_y',
 'MSA',
 'PMSA',
 'CSA',
 'CBSA',
 'CBSA_DIV',
 'NumberOfBusinesses',
 'NumberOfEmployees',
 'BusinessFirstQuarterPayroll',
 'BusinessAnnualPayroll',
 'BusinessEmploymentFlag',
 'GrowthRank',
 'GrowthHousingUnits2009',
 'GrowthHousingUnits2010',
 'GrowthIncreaseNumber',
 'GrowthIncreasePercentage',
 'CBSAPop2003',
 'CBSADivPop2003',
 'DeliveryResidential',
 'DeliveryBusiness',
 'DeliveryTotal',
 'PopulationEstimate',
 'LandArea',
 'WaterArea',
 'CityAliasCode',
 'BoxCount',
 'SFDU',
 'MFDU',
 'StateANSI',
 '

In [43]:
stat1 = df4[continous_list].apply(lambda x:sum(x.isnull()==False)/len(x))
k1 = pd.DataFrame({'name':list(stat1.index),'value':stat1})

In [44]:
k1.sort_values(by='value',ascending=False)

,name,value
label,label,1.000000
Transactions,Transactions,0.999998
CloseDt,CloseDt,0.999542
CBSA,CBSA,0.993979
PopulationEstimate,PopulationEstimate,0.993979
TimeZone_y,TimeZone_y,0.993979
FinanceNumber,FinanceNumber,0.993979
CountyANSI,CountyANSI,0.993979
StateANSI,StateANSI,0.993979
NumberOfBusinesses,NumberOfBusinesses,0.993979


In [45]:
keep_continuous = list(k1[k1['value']>0.9]['name'])

In [46]:
drop1 = list(k1[k1['value']<0.9]['name'])

In [47]:
drop1

['SellingSqFt',
 'Type',
 'BusinessEmploymentFlag',
 'GrowthHousingUnits2009',
 'GrowthHousingUnits2010',
 'CBSAPop2003',
 'CBSADivPop2003',
 'CityAliasCode',
 'UniqueZIPName']

In [48]:
keep_continuous

['Transactions',
 'CloseDt',
 'Population',
 'HouseholdsPerZipcode',
 'WhitePopulation',
 'BlackPopulation',
 'HispanicPopulation',
 'AsianPopulation',
 'HawaiianPopulation',
 'IndianPopulation',
 'OtherPopulation',
 'MalePopulation',
 'FemalePopulation',
 'PersonsPerHousehold',
 'AverageHouseValue',
 'IncomePerHousehold',
 'MedianAge',
 'MedianAgeMale',
 'MedianAgeFemale',
 'Latitude',
 'Longitude',
 'Elevation',
 'CountyFIPS',
 'StateFIPS',
 'TimeZone_y',
 'MSA',
 'PMSA',
 'CSA',
 'CBSA',
 'CBSA_DIV',
 'NumberOfBusinesses',
 'NumberOfEmployees',
 'BusinessFirstQuarterPayroll',
 'BusinessAnnualPayroll',
 'GrowthRank',
 'GrowthIncreaseNumber',
 'GrowthIncreasePercentage',
 'DeliveryResidential',
 'DeliveryBusiness',
 'DeliveryTotal',
 'PopulationEstimate',
 'LandArea',
 'WaterArea',
 'BoxCount',
 'SFDU',
 'MFDU',
 'StateANSI',
 'CountyANSI',
 'FinanceNumber',
 'label']

In [49]:
df5.drop(drop1,axis=1,inplace=True)
for x in keep_continuous:
    df5[x] = df5[x].fillna(-1)

In [50]:
df5.to_csv('/Users/tianyizhao/Desktop/PUMA Dataset/mid_result1.csv',index=0)